<a href="https://colab.research.google.com/github/serereuk/Information_retrieval/blob/master/textCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# textCNN Rand

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd '/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata'

/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata


In [0]:
import re
import tensorflow as tf
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Dropout, GlobalMaxPooling1D, ReLU

In [0]:
def Reader(text_file_path):
    data = [];
    for i in text_file_path:
        data += open(i, 'r', encoding="ISO-8859-1").readlines()
    label = [0] * int(len(data)/2) + [1] * int(len(data)/2)
    for i in range(len(data)):
        data[i] = re.sub('^\W+', "", data[i].lower().strip())
    return data, label

In [0]:
def Indexing_padding(data, tok=None):
    if tok == None:
        tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
        tokenizer.fit_on_texts(data)
    else:
        tokenizer = tok
    tensor = tokenizer.texts_to_sequences(data)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, tokenizer

In [0]:
data, y = Reader(['rt-polarity.neg.txt', 'rt-polarity.pos.txt'])
tensor, tokenizer = Indexing_padding(data, None)
x_train, x_val, y_train, y_val = train_test_split(tensor, y, test_size=0.1, stratify=y, random_state=0)

In [0]:
Buffer_size = len(x_train) + 1
vocab_size = len(tokenizer.word_index) + 1
Batch_size = 50
epoch = 10
learning_rate = 0.001
dimension = 300
tf.random.set_seed(1)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(Buffer_size)
train_dataset = train_dataset.batch(Batch_size, drop_remainder=False)

test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(Buffer_size)
test_dataset = test_dataset.batch(Batch_size, drop_remainder=False)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
class textCNN(Model):

    def __init__(self, vocab_size, dimension=300):
        super(textCNN, self).__init__()
        self.embed = Embedding(vocab_size, dimension)
        self.conv1 = Conv1D(100, 3, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.conv2 = Conv1D(100, 4, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.conv3 = Conv1D(100, 5, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.dense = Dense(2, activation='softmax', kernel_constraint=max_norm(3))

    def call(self, x):
        x = self.embed(x)
        filter_result1 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv1(x))))
        filter_result2 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv2(x))))
        filter_result3 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv3(x))))
        x = tf.concat([filter_result1, filter_result2, filter_result3], axis=1)
        x = self.dense(x)
        return x

model = textCNN(vocab_size, dimension)

In [0]:
@tf.function
def train_step(x, label):
    with tf.GradientTape()as tape:
        predictions = model(x)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(label, predictions)

In [0]:
@tf.function
def test_step(x, label):
    predictions = model(x, training=False)
    loss = loss_object(label, predictions)
    test_loss(loss)
    test_accuracy(label, predictions)

In [0]:
for ep in range(epoch):
    for x, label in train_dataset:
        train_step(x,label)
    for x, label in test_dataset:
        test_step(x, label)
    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(ep+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.6140450239181519, 정확도: 67.8895263671875, 테스트 손실: 0.5566848516464233, 테스트 정확도: 73.66448211669922
에포크: 2, 손실: 0.5235644578933716, 정확도: 78.15007781982422, 테스트 손실: 0.5576726794242859, 테스트 정확도: 73.52389526367188
에포크: 3, 손실: 0.46739476919174194, 정확도: 84.16883850097656, 테스트 손실: 0.5549745559692383, 테스트 정확도: 74.03936767578125
에포크: 4, 손실: 0.43333637714385986, 정확도: 87.74882507324219, 테스트 손실: 0.555284321308136, 테스트 정확도: 74.203369140625
에포크: 5, 손실: 0.411521315574646, 정확도: 89.99896240234375, 테스트 손실: 0.555732786655426, 테스트 정확도: 74.24554443359375
에포크: 6, 손실: 0.39659932255744934, 정확도: 91.52857971191406, 테스트 손실: 0.5551154613494873, 테스트 정확도: 74.24242401123047
에포크: 7, 손실: 0.3857688903808594, 정확도: 92.63455963134766, 테스트 손실: 0.5548070073127747, 테스트 정확도: 74.30713653564453
에포크: 8, 손실: 0.37750542163848877, 정확도: 93.47837829589844, 테스트 손실: 0.5554026961326599, 테스트 정확도: 74.34395599365234
에포크: 9, 손실: 0.3710392713546753, 정확도: 94.13699340820312, 테스트 손실: 0.5560609698295593, 테스트 정확도: 74.23721313476562
에포크

# textCNN_static

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd '/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata'

/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata


In [0]:
import re
import tensorflow as tf
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Dropout, GlobalMaxPooling1D, ReLU

In [0]:
def Reader(text_file_path):
    data = [];
    for i in text_file_path:
        data += open(i, 'r', encoding="ISO-8859-1").readlines()
    label = [0] * int(len(data)/2) + [1] * int(len(data)/2)
    for i in range(len(data)):
        data[i] = re.sub('^\W+', "", data[i].lower().strip())
    return data, label

In [0]:
def Indexing_padding(data, tok=None):
    if tok == None:
        tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
        tokenizer.fit_on_texts(data)
    else:
        tokenizer = tok
    tensor = tokenizer.texts_to_sequences(data)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, tokenizer

In [0]:
data, y = Reader(['rt-polarity.neg.txt', 'rt-polarity.pos.txt'])
tensor, tokenizer = Indexing_padding(data, None)
x_train, x_val, y_train, y_val = train_test_split(tensor, y, test_size=0.1, stratify=y, random_state=0)

In [0]:
Buffer_size = len(x_train) + 1
vocab_size = len(tokenizer.word_index) + 1
Batch_size = 50
epoch = 10
learning_rate = 0.001
dimension = 300
tf.random.set_seed(1)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(Buffer_size)
train_dataset = train_dataset.batch(Batch_size, drop_remainder=False)

test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(Buffer_size)
test_dataset = test_dataset.batch(Batch_size, drop_remainder=False)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/information/CNN, RNN/CNN/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
var = tf.Variable(tf.initializers.RandomUniform(-0.001, 0.001)(shape=(vocab_size, 300))).numpy()
i = 0
for index, word in zip(list(tokenizer.word_index.values()), list(tokenizer.word_index.keys())):
    try:
        var[index] = model.get_vector(word)
        i += 1
    except:
        pass
print(i, len(var))

15902 21374


In [0]:
del model

In [0]:
class textCNN_static(Model):

    def __init__(self, vocab_size, pretrained, dimension=300):
        super(textCNN_static, self).__init__()
        self.embed = Embedding(vocab_size, dimension, embeddings_initializer=tf.keras.initializers.Constant(pretrained),trainable=False)
        self.conv1 = Conv1D(100, 3, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.conv2 = Conv1D(100, 4, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.conv3 = Conv1D(100, 5, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.dense = Dense(2, activation='softmax', kernel_constraint=max_norm(3))

    def call(self, x):
        x = self.embed(x)
        filter_result1 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv1(x))))
        filter_result2 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv2(x))))
        filter_result3 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv3(x))))
        x = tf.concat([filter_result1, filter_result2, filter_result3], axis=1)
        x = self.dense(x)
        return x

model = textCNN_static(vocab_size,var, dimension)

In [0]:
@tf.function
def train_step(x, label):
    with tf.GradientTape()as tape:
        predictions = model(x)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(label, predictions)

In [0]:
@tf.function
def test_step(x, label):
    predictions = model(x, training=False)
    loss = loss_object(label, predictions)
    test_loss(loss)
    test_accuracy(label, predictions)

In [0]:
for ep in range(epoch):
    for x, label in train_dataset:
        train_step(x,label)
    for x, label in test_dataset:
        test_step(x, label)
    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(ep+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.5627406239509583, 정확도: 74.27827453613281, 테스트 손실: 0.5388481616973877, 테스트 정확도: 75.72633361816406
에포크: 2, 손실: 0.5182666778564453, 정확도: 79.3434066772461, 테스트 손실: 0.5322895050048828, 테스트 정확도: 76.42924499511719
에포크: 3, 손실: 0.48660263419151306, 정확도: 83.15094757080078, 테스트 손실: 0.5271875858306885, 테스트 정확도: 76.6323013305664
에포크: 4, 손실: 0.46000170707702637, 정확도: 86.12037658691406, 테스트 손실: 0.52436363697052, 테스트 정확도: 77.10871887207031
에포크: 5, 손실: 0.43882647156715393, 정확도: 88.33142852783203, 테스트 손실: 0.5227517485618591, 테스트 정확도: 77.33833312988281
에포크: 6, 손실: 0.4223492741584778, 정확도: 89.96178436279297, 테스트 손실: 0.520948588848114, 테스트 정확도: 77.6788558959961
에포크: 7, 손실: 0.4096299111843109, 정확도: 91.17546081542969, 테스트 손실: 0.5200791358947754, 테스트 정확도: 77.82836151123047
에포크: 8, 손실: 0.39963528513908386, 정확도: 92.1143798828125, 테스트 손실: 0.5188255906105042, 테스트 정확도: 77.96392059326172
에포크: 9, 손실: 0.39166054129600525, 정확도: 92.85623168945312, 테스트 손실: 0.518675684928894, 테스트 정확도: 78.04853057861328
에포크:

# textCNN_non_static

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd '/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata'

/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata


In [0]:
import re
import tensorflow as tf
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Dropout, GlobalMaxPooling1D, ReLU

In [0]:
def Reader(text_file_path):
    data = [];
    for i in text_file_path:
        data += open(i, 'r', encoding="ISO-8859-1").readlines()
    label = [0] * int(len(data)/2) + [1] * int(len(data)/2)
    for i in range(len(data)):
        data[i] = re.sub('^\W+', "", data[i].lower().strip())
    return data, label

In [0]:
def Indexing_padding(data, tok=None):
    if tok == None:
        tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
        tokenizer.fit_on_texts(data)
    else:
        tokenizer = tok
    tensor = tokenizer.texts_to_sequences(data)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, tokenizer

In [0]:
data, y = Reader(['rt-polarity.neg.txt', 'rt-polarity.pos.txt'])
tensor, tokenizer = Indexing_padding(data, None)
x_train, x_val, y_train, y_val = train_test_split(tensor, y, test_size=0.1, stratify=y, random_state=0)

In [0]:
Buffer_size = len(x_train) + 1
vocab_size = len(tokenizer.word_index) + 1
Batch_size = 50
epoch = 10
learning_rate = 0.001
dimension = 300
tf.random.set_seed(1)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(Buffer_size)
train_dataset = train_dataset.batch(Batch_size, drop_remainder=False)

test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(Buffer_size)
test_dataset = test_dataset.batch(Batch_size, drop_remainder=False)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/information/CNN, RNN/CNN/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
var = tf.Variable(tf.initializers.RandomUniform(-0.001, 0.001)(shape=(vocab_size, 300))).numpy()
i = 0
for index, word in zip(list(tokenizer.word_index.values()), list(tokenizer.word_index.keys())):
    try:
        var[index] = model.get_vector(word)
        i += 1
    except:
        pass
print(i, len(var))

15902 21374


In [0]:
del model

In [0]:
class textCNN_non_static(Model):

    def __init__(self, vocab_size, pretrained, dimension=300):
        super(textCNN_non_static, self).__init__()
        self.embed = Embedding(vocab_size, dimension, embeddings_initializer=tf.keras.initializers.Constant(pretrained),trainable=True)
        self.conv1 = Conv1D(100, 3, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.conv2 = Conv1D(100, 4, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.conv3 = Conv1D(100, 5, strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1]))
        self.dense = Dense(2, activation='softmax', kernel_constraint=max_norm(3))

    def call(self, x):
        x = self.embed(x)
        filter_result1 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv1(x))))
        filter_result2 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv2(x))))
        filter_result3 = Dropout(0.5)(GlobalMaxPooling1D()(ReLU()(self.conv3(x))))
        x = tf.concat([filter_result1, filter_result2, filter_result3], axis=1)
        x = self.dense(x)
        return x

model = textCNN_non_static(vocab_size,var, dimension)

In [0]:
@tf.function
def train_step(x, label):
    with tf.GradientTape()as tape:
        predictions = model(x)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(label, predictions)

In [0]:
@tf.function
def test_step(x, label):
    predictions = model(x, training=False)
    loss = loss_object(label, predictions)
    test_loss(loss)
    test_accuracy(label, predictions)

In [0]:
for ep in range(epoch):
    for x, label in train_dataset:
        train_step(x,label)
    for x, label in test_dataset:
        test_step(x, label)
    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(ep+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.5502836108207703, 정확도: 75.14330291748047, 테스트 손실: 0.5157164335250854, 테스트 정확도: 78.912841796875
에포크: 2, 손실: 0.48189282417297363, 정확도: 82.87649536132812, 테스트 손실: 0.5163614153862, 테스트 정확도: 78.53795623779297
에포크: 3, 손실: 0.43908894062042236, 정확도: 87.52822875976562, 테스트 손실: 0.5117866396903992, 테스트 정확도: 78.912841796875
에포크: 4, 손실: 0.4124760627746582, 정확도: 90.24231719970703, 테스트 손실: 0.5113073587417603, 테스트 정확도: 78.93627166748047
에포크: 5, 손실: 0.3949950039386749, 정확도: 91.98957824707031, 테스트 손실: 0.5102297067642212, 테스트 정확도: 79.08153533935547
에포크: 6, 손실: 0.38281187415122986, 정확도: 93.19437408447266, 테스트 손실: 0.50956791639328, 테스트 정확도: 78.92845916748047
에포크: 7, 손실: 0.3738698959350586, 정확도: 94.07429504394531, 테스트 손실: 0.5091556310653687, 테스트 정확도: 78.92623138427734
에포크: 8, 손실: 0.3670634925365448, 정확도: 94.74205017089844, 테스트 손실: 0.5089240670204163, 테스트 정확도: 78.901123046875
에포크: 9, 손실: 0.3617025315761566, 정확도: 95.2672119140625, 테스트 손실: 0.5088534951210022, 테스트 정확도: 78.92325592041016
에포크: 10, 손

# textCNN_multichannel


In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata'

/content/drive/My Drive/information/CNN, RNN/CNN/rt-polaritydata(MR)/rt-polaritydata


In [0]:
import re
import tensorflow as tf
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Embedding, Conv2D, Dense, Dropout, GlobalMaxPooling2D, ReLU

In [0]:
def Reader(text_file_path):
    data = [];
    for i in text_file_path:
        data += open(i, 'r', encoding="ISO-8859-1").readlines()
    label = [0] * int(len(data)/2) + [1] * int(len(data)/2)
    for i in range(len(data)):
        data[i] = re.sub('^\W+', "", data[i].lower().strip())
    return data, label

In [0]:
def Indexing_padding(data, tok=None):
    if tok == None:
        tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
        tokenizer.fit_on_texts(data)
    else:
        tokenizer = tok
    tensor = tokenizer.texts_to_sequences(data)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, tokenizer

In [0]:
data, y = Reader(['rt-polarity.neg.txt', 'rt-polarity.pos.txt'])
tensor, tokenizer = Indexing_padding(data, None)
x_train, x_val, y_train, y_val = train_test_split(tensor, y, test_size=0.1, stratify=y, random_state=0)

In [0]:
Buffer_size = len(x_train) + 1
vocab_size = len(tokenizer.word_index) + 1
Batch_size = 50
epoch = 10
learning_rate = 0.001
dimension = 300
tf.random.set_seed(1)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(Buffer_size)
train_dataset = train_dataset.batch(Batch_size, drop_remainder=False)

test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(Buffer_size)
test_dataset = test_dataset.batch(Batch_size, drop_remainder=False)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [10]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/information/CNN, RNN/CNN/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
var = tf.Variable(tf.initializers.RandomUniform(-0.001, 0.001)(shape=(vocab_size, 300))).numpy()
i = 0
for index, word in zip(list(tokenizer.word_index.values()), list(tokenizer.word_index.keys())):
    try:
        var[index] = model.get_vector(word)
        i += 1
    except:
        pass
print(i, len(var))

15902 21374


In [0]:
del model

In [0]:
class textCNN_multi(Model):

    def __init__(self, vocab_size, pretrained, dimension=300):
        super(textCNN_multi, self).__init__()
        self.embed = Embedding(vocab_size, dimension, embeddings_initializer=tf.keras.initializers.Constant(pretrained),trainable=True)
        self.embed2 = Embedding(vocab_size, dimension, embeddings_initializer=tf.keras.initializers.Constant(pretrained),trainable=False)
        self.conv1 = Conv2D(100, (3, dimension), strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1,2]))
        self.conv2 = Conv2D(100, (4, dimension), strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1,2]))
        self.conv3 = Conv2D(100, (5, dimension), strides=1, padding='valid', kernel_constraint=max_norm(3, axis=[0,1,2]))
        self.dense = Dense(2, activation='softmax', kernel_constraint=max_norm(3))

    def call(self, x):
        x1 = self.embed(x)
        x2 = self.embed2(x)
        x = tf.stack([x1, x2], axis=3)
        filter_result1 = Dropout(0.5)(GlobalMaxPooling2D()(ReLU()(self.conv1(x))))
        filter_result2 = Dropout(0.5)(GlobalMaxPooling2D()(ReLU()(self.conv2(x))))
        filter_result3 = Dropout(0.5)(GlobalMaxPooling2D()(ReLU()(self.conv3(x))))
        x = tf.concat([filter_result1, filter_result2, filter_result3], axis=1)
        x = self.dense(x)
        return x

model = textCNN_multi(vocab_size, var, dimension)

In [0]:
@tf.function
def train_step(x, label):
    with tf.GradientTape()as tape:
        predictions = model(x)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(label, predictions)

In [0]:
@tf.function
def test_step(x, label):
    predictions = model(x, training=False)
    loss = loss_object(label, predictions)
    test_loss(loss)
    test_accuracy(label, predictions)

In [31]:
for ep in range(epoch):
    for x, label in train_dataset:
        train_step(x,label)
    for x, label in test_dataset:
        test_step(x, label)
    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(ep+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.5381555557250977, 정확도: 76.0500259399414, 테스트 손실: 0.5221347212791443, 테스트 정확도: 76.38237762451172
에포크: 2, 손실: 0.48139986395835876, 정확도: 82.42314147949219, 테스트 손실: 0.514563798904419, 테스트 정확도: 77.6476058959961
에포크: 3, 손실: 0.441375732421875, 정확도: 86.77088928222656, 테스트 손실: 0.5135930776596069, 테스트 정확도: 77.81942749023438
에포크: 4, 손실: 0.4157598912715912, 정확도: 89.48670959472656, 테스트 손실: 0.5127797722816467, 테스트 정확도: 78.1396484375
에포크: 5, 손실: 0.3986738622188568, 정확도: 91.27045440673828, 테스트 손실: 0.512876570224762, 테스트 정확도: 78.29428100585938
에포크: 6, 손실: 0.38665318489074707, 정확도: 92.51693725585938, 테스트 손실: 0.5133378505706787, 테스트 정확도: 78.27241516113281
에포크: 7, 손실: 0.3777911365032196, 정확도: 93.42961120605469, 테스트 손실: 0.5142425894737244, 테스트 정확도: 78.27018737792969
에포크: 8, 손실: 0.370988130569458, 정확도: 94.12715148925781, 테스트 손실: 0.5148140788078308, 테스트 정확도: 78.23336791992188
에포크: 9, 손실: 0.3655976951122284, 정확도: 94.67894744873047, 테스트 손실: 0.515252947807312, 테스트 정확도: 78.24637603759766
에포크: 10, 손